In [1]:
import pandas as pd

Before we begin, I noticed that rows 7422 to 7699 in the original raw data CSV that you sent me contains random information that's not relevant to this data set (things like notes about where the data was from and when the report was produced). This shouldn't be included in the data set. I'd recommend deleting those lines before importing it into pandas. The data that is imported in the next line has had these lines removed.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jennynz/co2-emissions/master/annual_ghg_data.csv', index_col=0)

# Australia 1990 vs 2000

To start with, let's just look at Australia's change in CO2 emissions from 1990 to 2000, and why there were issues with subtracting the columns. This will demonstrate the importance of data cleaning (and also how most of data science ends up being cleaning and validating your data!)

In [3]:
# Have a quick look at the first 10 rows
df.head(10)

,Party,Sector,Year,CO2 emissions (kt)
0,Australia,Energy,1990,259089.0635
1,Australia,Energy,1991,261077.8943
2,Australia,Energy,1992,265636.3056
3,Australia,Energy,1993,269692.6933
4,Australia,Energy,1994,272493.5233
5,Australia,Energy,1995,283546.7993
6,Australia,Energy,1996,290186.9462
7,Australia,Energy,1997,298419.5558
8,Australia,Energy,1998,311658.248
9,Australia,Energy,1999,320154.8441


In [4]:
# Get the subsets for Australia 1990 and 2000
au1990 = df[(df['Party'] == 'Australia') & (df['Year'] == '1990')]
au2000 = df[(df['Party'] == 'Australia') & (df['Year'] == '2000')]

In [5]:
# Subtract the CO2 emissions of the two subsets from each other
au2000['CO2 emissions (kt)'] - au1990['CO2 emissions (kt)']

0      NaN
10     NaN
28     NaN
38     NaN
56     NaN
66     NaN
84     NaN
94     NaN
112    NaN
122    NaN
140    NaN
150    NaN
Name: CO2 emissions (kt), dtype: object

## Data cleaning

Oh no, we're getting `NaN` (not a number) values for everything! Let's see what's going on...


### Setting the index to something more useful

In [6]:
au1990

,Party,Sector,Year,CO2 emissions (kt)
0,Australia,Energy,1990,259089.0635
28,Australia,Industrial Processes and Product Use,1990,18679.93852
56,Australia,Agriculture,1990,582.0132094
84,Australia,Land Use LandUse Change and Forestry,1990,165522.8783
112,Australia,Waste,1990,73.36044861
140,Australia,Other,1990,NO


In [7]:
au2000

,Party,Sector,Year,CO2 emissions (kt)
10,Australia,Energy,2000,326738.2134
38,Australia,Industrial Processes and Product Use,2000,21727.58414
66,Australia,Agriculture,2000,1701.121427
94,Australia,Land Use LandUse Change and Forestry,2000,32667.3159
122,Australia,Waste,2000,27.6627884
150,Australia,Other,2000,NO


Okay, looks like the indexes are all different. When you try to subtract one column from another, it's going to figure out which row to subtract from which based on their indexes.

There are lots of ways we could do this.

1. Reset indexes

Fortunately, the `Sector` values are in the same order between the two datasets. We can just re-index both subsets (basically get it to re-number the rows from the top) and it should do the trick. You can read about the pandas reset_index method in their [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html).

2. Set the `Sector` column as the index

Since we already know that the data in these subsets are all from Australia from a particular year, the column that's really distinguishing the data is the `Sector`. We can set `Sector` as the index with [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html).

In [8]:
au1990.set_index('Sector', inplace=True)
au2000.set_index('Sector', inplace=True)

In [9]:
au1990

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,1990,259089.0635
Industrial Processes and Product Use,Australia,1990,18679.93852
Agriculture,Australia,1990,582.0132094
Land Use LandUse Change and Forestry,Australia,1990,165522.8783
Waste,Australia,1990,73.36044861
Other,Australia,1990,NO


In [10]:
au2000

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,2000,326738.2134
Industrial Processes and Product Use,Australia,2000,21727.58414
Agriculture,Australia,2000,1701.121427
Land Use LandUse Change and Forestry,Australia,2000,32667.3159
Waste,Australia,2000,27.6627884
Other,Australia,2000,NO


### Cleaning the CO2 emissions column

We can now see that the indexes match up - `Energy` is on row 0, `Waste` on row 4 etc. But what's this `'NO'` that we're getting in the `Other` sector? Let's dig a bit deeper by looking at the original data (which is still stored in `df`).


In [11]:
df[(df['Party'] == 'Australia') & (df['Sector'] == 'Other')]

,Party,Sector,Year,CO2 emissions (kt)
140,Australia,Other,1990,NO
141,Australia,Other,1991,NO
142,Australia,Other,1992,NO
143,Australia,Other,1993,NO
144,Australia,Other,1994,NO
145,Australia,Other,1995,NO
146,Australia,Other,1996,NO
147,Australia,Other,1997,NO
148,Australia,Other,1998,NO
149,Australia,Other,1999,NO


Wow, weird! Looks like whoever collected this data, when they didn't have any data for a particular sector (like the `Other` sector in this case), they just typed a string saying `'NO'` in there. That's making it difficult for pandas to interpret the emissions column as a numeric value, because there are letters in there.

In data science, we want to be really explicit when there's data missing, so in cases like these we put a `NaN` to tell others that there was no data for this point.

In [12]:
# Check if there are any other rows other than Australia/Other that has non-numeric values
df[df['CO2 emissions (kt)'].str.isnumeric()]

ValueError: cannot index with vector containing NA / NaN values

Ugh, there are some NaNs in the emissions column too. Since it's "not a number", pandas can't say for sure whether it is numeric or not, so it just returns an error. Looks like the CO2 emissions column needs a bit of work.

In [13]:
# For each row, try coercing it into a `float` type.
# If it fails, then fill it with `None` so that we know it's purposefully empty - there was no data there in the first place.
# This will be turned into `NaN` by pandas.
def validate_numeric(x):
    try:
        x = pd.to_numeric(x)
    except: 
        x = None
    return x

df['CO2 emissions (kt)'] = df['CO2 emissions (kt)'].map(validate_numeric)

In [14]:
df['CO2 emissions (kt)']

0       259089.0635
1       261077.8943
2       265636.3056
3       269692.6933
4       272493.5233
           ...     
7415            NaN
7416            NaN
7417            NaN
7418            NaN
7419            NaN
Name: CO2 emissions (kt), Length: 7420, dtype: float64

Sweet, that looks pretty good! Maybe we should check explicitly to see if all the `'NO'` values are gone.

In [15]:
# This assert method throws an error if the condition inside it is False.
# Here, we're checking that the number of rows where CO2 emissions is 'NO' is 0.
rows_with_co2_emissions_as_NO = df[df['CO2 emissions (kt)'] == 'NO']
number_of_rows_with_co2_emissions_as_NO = len(rows_with_co2_emissions_as_NO)
assert(number_of_rows_with_co2_emissions_as_NO == 0)

# That's a bit long and verbose, we could do it in one line without giving up too much readability:
assert(len(df[df['CO2 emissions (kt)'] == 'NO']) == 0)
# No errors = Yeap, all the 'NO's are gone!

//anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Cleaning the 'Year' column

We're almost done, but I noticed something when I had a flick through the raw CSV file. I noticed that the last year for each sector (2017) always had this big string that said `'Last Inventory Year (2017)'` instead of just the number `2017`. We'll have to clean the data a bit to get rid of that if we want to be able to use the data from 2017.

In [16]:
# We have a nice dataset of 275 data points per year for all the years.
# The only gross bit is that the name of theyear `2017` is a big long string rather than just the number like all the others.
df['Year'].value_counts().sort_index()

1990                          265
1991                          265
1992                          265
1993                          265
1994                          265
1995                          265
1996                          265
1997                          265
1998                          265
1999                          265
2000                          265
2001                          265
2002                          265
2003                          265
2004                          265
2005                          265
2006                          265
2007                          265
2008                          265
2009                          265
2010                          265
2011                          265
2012                          265
2013                          265
2014                          265
2015                          265
2016                          265
Last Inventory Year (2017)    265
Name: Year, dtype: int64

In [17]:
# Replace weird last year string with the number `2017`
df['Year'].replace(to_replace='Last Inventory Year (2017)', value='2017', inplace=True)

### Validating your results

In [18]:
# Make the subsets again from the cleaned data, reset the indexes, and try the subtraction again
au1990 = df[(df['Party'] == 'Australia') & (df['Year'] == '1990')]
au2000 = df[(df['Party'] == 'Australia') & (df['Year'] == '2000')]
au1990.set_index('Sector', inplace=True)
au2000.set_index('Sector', inplace=True)

au_1990_2000_diff = au2000['CO2 emissions (kt)'] - au1990['CO2 emissions (kt)']
au_1990_2000_diff

Sector
Energy                                   67649.149900
Industrial Processes and Product Use      3047.645620
Agriculture                               1119.108218
Land Use LandUse Change and Forestry   -132855.562400
Waste                                      -45.697660
Other                                             NaN
Name: CO2 emissions (kt), dtype: float64

Yay, it works! We've got that `NaN` in the last row because that's for the `Other` sector, which didn't have any data (just the weird `'NO'` values).

Maybe we should add a quick test to check that the numbers are correct. Let's check that the difference in the `Energy` sector is indeed equal to the value from 2000 minus the value from 1990.

In [19]:
au1990

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,1990,259089.063500
Industrial Processes and Product Use,Australia,1990,18679.938520
Agriculture,Australia,1990,582.013209
Land Use LandUse Change and Forestry,Australia,1990,165522.878300
Waste,Australia,1990,73.360449
Other,Australia,1990,NaN


In [20]:
au1990_energy = au1990.loc['Energy', 'CO2 emissions (kt)']
au1990_energy

259089.0635

In [21]:
au2000

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,2000,326738.213400
Industrial Processes and Product Use,Australia,2000,21727.584140
Agriculture,Australia,2000,1701.121427
Land Use LandUse Change and Forestry,Australia,2000,32667.315900
Waste,Australia,2000,27.662788
Other,Australia,2000,NaN


In [22]:
au2000_energy = au2000.loc['Energy', 'CO2 emissions (kt)']
au2000_energy

326738.2134

In [23]:
# No error = assertion passed!
assert(au2000_energy - au1990_energy == au_1990_2000_diff['Energy'])

## Putting it all together

You will have noticed that there was a fair bit of repetition as we went through the data and found dirty bits that we had to clean. Let's put what we've done into functions so that we can re-use them easily without re-writing code.

In [24]:
def validate_numeric(x):
    try:
        x = pd.to_numeric(x)
    except: 
        x = None
    return x

In [25]:
def clean_ghg_data(df):
    df['CO2 emissions (kt)'] = df['CO2 emissions (kt)'].map(validate_numeric)
    df['Year'].replace(to_replace='Last Inventory Year (2017)', value=2017, inplace=True)
    return df

In [26]:
def get_data_for_country_and_year(df, country, year):
    data = df[(df['Party'] == country) & (df['Year'] == year)]
    data.set_index('Sector', inplace=True)
    return data

In [27]:
def get_net_co2_difference(start_df, end_df):
    return start_df['CO2 emissions (kt)'] - end_df['CO2 emissions (kt)']

In [28]:
# From the top!
df = pd.read_csv('Annual_GHG_data_reshaped.csv', index_col=0)

In [29]:
df = clean_ghg_data(df)

In [30]:
au1990 = get_data_for_country_and_year(df, 'Australia', '1990')
au1990

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,1990,259089.063500
Industrial Processes and Product Use,Australia,1990,18679.938520
Agriculture,Australia,1990,582.013209
Land Use LandUse Change and Forestry,Australia,1990,165522.878300
Waste,Australia,1990,73.360449
Other,Australia,1990,NaN


In [31]:
au2000 = get_data_for_country_and_year(df, 'Australia', '2000')
au2000

,Party,Year,CO2 emissions (kt)
Sector,,,
Energy,Australia,2000,326738.213400
Industrial Processes and Product Use,Australia,2000,21727.584140
Agriculture,Australia,2000,1701.121427
Land Use LandUse Change and Forestry,Australia,2000,32667.315900
Waste,Australia,2000,27.662788
Other,Australia,2000,NaN


In [32]:
get_net_co2_difference(au1990, au2000)

Sector
Energy                                  -67649.149900
Industrial Processes and Product Use     -3047.645620
Agriculture                              -1119.108218
Land Use LandUse Change and Forestry    132855.562400
Waste                                       45.697660
Other                                             NaN
Name: CO2 emissions (kt), dtype: float64

# User interaction

We've got these neat functions which already take the country and year as inputs. It would only take a little more work to be able to ask the user for these. Something like the stuff below, in a Python script (a file with `.py` on the end), would be a (very simple and not-very-user-friendly) MVP version of such an interactive tool.

In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/jennynz/co2-emissions/master/annual_ghg_data.csv', index_col=0)
df = clean_ghg_data(df)

print('*** Net CO2 emissions difference calculator ***')
country = input('Enter a country: ') # e.g. Australia
start_year = input('Enter start year (between 1990 and 2017 inclusive): ') # e.g. 1990
end_year = input('Enter end year (between 1990 and 2017 inclusive): ') # e.g. 2000

# Some examples of checks you could do to stop users from entering inputs that won't work

# country_list = [str.lower(country) for country in df['Party'].unique()]
# assert(country in df['Party'].unique(), 'Country name not valid')
# assert(type(start_year) == int, 'Year must be an integer')
# assert(type(end_year) == int, 'Year must be an integer')
# assert((start_year >= 1990) & (start_year <= 2016), 'Please enter a year between 1990 and 2017 inclusive.')
# assert((start_year >= 1991) & (start_year <= 2017), 'Please enter a year between 1990 and 2017 inclusive.')
# assert(start_year < end_year, 'Start year must be before end year')

start = get_data_for_country_and_year(df, country, start_year)
end = get_data_for_country_and_year(df, country, end_year)

print(f'\n*** Results for {country} between {start_year}-{end_year} ***')
get_net_co2_difference(start, end)

*** Net CO2 emissions difference calculator ***
Enter a country: Australia
Enter start year (between 1990 and 2017 inclusive): 1992
Enter end year (between 1990 and 2017 inclusive): 2012
*** Results for Australia between 1992-2012 ***


Sector
Energy                                 -117873.768900
Industrial Processes and Product Use     -2889.502310
Agriculture                              -1355.262504
Land Use LandUse Change and Forestry     78572.831121
Waste                                       43.673097
Other                                             NaN
Name: CO2 emissions (kt), dtype: float64

# Plots for years in given range

Instead of just getting certain years, pandas makes it easy to do bulk operations on a whole lot of rows of data.
You could modify/add to the code above to get it to return all the years (maybe within a range that the user inputs) and sectors (maybe also input/checkboxed depending on what the user wants) for a given country, and then plot that on a line graph with different colours for each sector.

The same would be possible for comparing countries for the same sector (e.g. compare how CO2 emissions have changed across time for Australia vs New Zealand vs USA, in the Energy sector).